In [0]:
%pip install meteostat tqdm
#%restart_python

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
import pandas as pd
from datetime import datetime
from meteostat import Point, Daily
from tqdm import tqdm # Para una barra de progreso, ¡muy útil!

In [0]:
df_con_geo = spark.table("workspace.default.complete_dataset_test")

In [0]:
# --- PASO 2: IDENTIFICAR LAS CONSULTAS ÚNICAS A LA API ---
# Esto es para ser eficientes. En lugar de una llamada por cada fila,
# hacemos una llamada por cada combinación única de lugar y fecha.
print("Paso 2: Identificando combinaciones únicas de latitud, longitud y fecha...")
loc_dates_unicas_pd = df_con_geo.select(
    "lat_municipio",
    "lon_municipio",
    "fecha_notificacion"
).distinct().toPandas()

# Limpiamos filas donde falte algún dato clave para la API
loc_dates_unicas_pd = loc_dates_unicas_pd.dropna(
    subset=['lat_municipio', 'lon_municipio', 'fecha_notificacion']
)

print(f"Se procesarán {len(loc_dates_unicas_pd)} combinaciones únicas.")

In [0]:
# --- PASO 3: LLAMAR A LA API EN BUCLE PARA OBTENER DATOS CLIMÁTICOS ---
print("Paso 3: Obteniendo datos climáticos desde la API de Meteostat...")

# ¡IMPORTANTE! Reemplaza esto con tu clave.
METEOSTAT_API_KEY = "8a4b224e05msh7db26cbb8f4532bp1b931djsn4a9215830d1a"
Daily.key = METEOSTAT_API_KEY

clima_data = [] # Aquí guardaremos los resultados

# Usamos tqdm para ver una barra de progreso durante el proceso
for index, row in tqdm(loc_dates_unicas_pd.iterrows(), total=loc_dates_unicas_pd.shape[0]):
    lat = row['lat_municipio']
    lon = row['lon_municipio']
    fecha = row['fecha_notificacion']
    
    # Creamos el punto geográfico para la API
    punto = Point(lat, lon)
    
    # Definimos el rango de un solo día
    start = end = fecha
    
    try:
        # Hacemos la llamada a la API
        data = Daily(punto, start, end).fetch()
        
        # Si la API devuelve datos para ese día...
        if not data.empty:
            # Extraemos la temperatura media y la precipitación
            temp_media = data.iloc[0].get('tavg', None) # Usamos .get para evitar errores si la columna no existe
            precipitacion = data.iloc[0].get('prcp', None)
            
            # Guardamos el resultado en nuestra lista
            clima_data.append({
                "lat_municipio": lat,
                "lon_municipio": lon,
                "fecha_notificacion": fecha,
                "temperatura_media": temp_media,
                "precipitacion": precipitacion
            })
    except Exception as e:
        # Si algo falla (ej. sin datos para esa estación), lo ignoramos y continuamos
        # print(f"No se pudo obtener datos para Lat:{lat}, Lon:{lon} en {fecha}. Error: {e}")
        pass

print(f"Se obtuvieron datos climáticos para {len(clima_data)} combinaciones.")

In [0]:
display(clima_data)

In [0]:
# --- PASO 4: UNIR LOS DATOS CLIMÁTICOS AL DATAFRAME PRINCIPAL ---
if clima_data:
    print("Paso 4: Uniendo los datos climáticos al DataFrame de Spark...")
    
    # Convertir la lista de resultados a un DataFrame de Pandas
    clima_pd = pd.DataFrame(clima_data)
    
    # Convertir el DataFrame de Pandas de vuelta a un DataFrame de Spark
    clima_spark_df = spark.createDataFrame(clima_pd)
    
    # Unir el DataFrame original con los nuevos datos climáticos
    # La clave de unión son las tres columnas que usamos para la consulta
    df_final_con_clima = df_con_geo.join(
        clima_spark_df,
        on=["lat_municipio", "lon_municipio", "fecha_notificacion"],
        how="left" # 'left' join para mantener todas las notificaciones originales
    )
    
    # --- PASO 5: GUARDAR EL DATASET FINAL ENRIQUECIDO ---
    print("Paso 5: Guardando la tabla final 'complete_dataset_con_clima'...")
    df_final_con_clima.write.mode("overwrite").saveAsTable("complete_dataset_con_clima")
    
    print("\n¡PROCESO COMPLETADO CON ÉXITO!")
    print("Se ha creado la tabla 'complete_dataset_con_clima'.")
    
    # Muestra una vista previa del resultado, filtrando para ver las filas que sí se enriquecieron
    display(df_final_con_clima.filter(col("temperatura_media").isNotNull()))

else:
    print("No se pudo obtener ningún dato climático. El DataFrame no ha sido modificado.")